In [1]:
!pip install pandas
!pip install openpyxl

  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
    --------------------------------------- 0.3/11.6 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.6 MB 2.8 MB/s eta 0:00:04
   ------- -------------------------------- 2.1/11.6 MB 3.7 MB/s eta 0:00:03
   ---------- ----------------------------- 3.1/11.6 MB 4.2 MB/s eta 0:00:03
   ---------------- ----------------------- 4.7/11.6 MB 4.9 MB/s eta 0:00:02
   --------------------- ------------------ 6.3/11.6 MB 5.3 MB/s eta 0:00:02
   ------------------------- -------------- 7.3/11.6 MB 5.3 MB/s eta 0:00:01
   ----------------------------- ---------- 8.7/11.6 MB 5.4 MB/s eta 0:00:01
   ----------------------------------- ---- 10.2/11.6 MB 5.5 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 5.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   --- --------------------

In [35]:
import openpyxl
import pandas as pd

# 엑셀 파일 경로
file_path = './CleanNetAddress.xlsx'

# 엑셀 파일에서 데이터 읽기
try:
    df = pd.read_excel(file_path, engine='openpyxl')
    print("엑셀 파일 읽기 성공")
except Exception as e:
    print(f"엑셀 파일 읽기 실패: {e}")

엑셀 파일 읽기 성공


In [36]:
import requests

# 엑셀 파일의 열 이름 확인
print("엑셀 파일의 열 이름:", df.columns)

# # 데이터 확인 (앞부분 5개 행 출력)
# print("엑셀 파일의 데이터 (앞부분 5개 행):")
# print(df.head())

# 주소 리스트 추출 (열 이름을 정확히 확인 후 수정하세요)
addresses = df['Address'].tolist()  # 'Address' 열 이름을 실제 엑셀 파일의 주소 열 이름으로 수정
print("추출된 주소 리스트:", addresses)

def geocode_address(address, api_key):
    base_url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"query": address}
    response = requests.get(base_url, headers=headers, params=params)
    response_json = response.json()
    if response_json['documents']:
        location = response_json['documents'][0]['address']
        return (address, location['y'], location['x'])  # 위도, 경도
    else:
        print(f"Geocoding failed for address {address}")
        return (address, None, None)

api_key = 'cac3864be84e85e6bc9f0497ec5ad86d'  # 여기에 실제 API 키를 입력하세요

# 각 주소를 지리적 좌표로 변환
coordinates = [geocode_address(address, api_key) for address in addresses if address is not None]

엑셀 파일의 열 이름: Index(['Address'], dtype='object')
추출된 주소 리스트: ['도안북로 158 공원 주차장', '도안북로 117번길 46 앞\n네거리변', '도안북로117번길 18 와\n도안북로117번길 24 사이', '대전 서구 도안동 896', '대전 서구 도안동 962', '대전 서구 도안동 922', '대전 서구 도안동 948', '동서대로 700 과\n대전 서구 도안동 857 사이', '대전 서구 도안동 877', '도안북로93번길 10-15 와\n도안북로93번길 10-11 사이', '도안북로93번길 20-11 과\n도안북로93번길 20-17 사이', '대전 서구 도안동 846 가운데\n(파스쿠찌와 올리브영 사이)', '대전 서구 동서대로 688\n(카카오공인중개사 앞 도로변)', '대전 유성구 용계동 664-6 가운데', '대전 유성구 용계동 665-5 가운데', '도안북로 73 과\n동서대로656번길 31-10 사이', '동서대로656번길 34-5 앞 도로변', '동서대로656번길 50 앞 도로변', '동서대로656번길 62 앞 도로변', '동서대로656번길 16-45 와\n동서대로656번길 34-16 사이', '도안대로 332 와\n모델하우스 사이', '대전 유성구 용계동 671-2 부지\n(도안북로 50-15 옆)', '대전 유성구 용계동 671-2 부지\n(도안북로 54-53 옆)', '대전 서구 도안동 1046', '대전 서구 도안동 1019 와\n원도안로241번길 14-31 사이', '원도안로241번길 24-19 와\n원도안로241번길 24-27 사이', '원도안로 257번길 48 뒤 산 밑', '원도안로241번길 11 과\n원도안로241번길 9 사이', '원도안로241번길 24-3 과\n공터 사이', '원도안로241번길 23-18 과\n원도안로241번길 23-14 사이', '원도안로242번길 15-3 앞 공원변', '원도안로224번안길 21 과\n원도안로 234 사이', '원도안로224번길 23-12 앞 

In [39]:
print("지리적 좌표:", coordinates)
print(len(coordinates))

coord_df = pd.DataFrame(coordinates)
coord_df.columns=['address', 'latitude', 'longitude']
print(coord_df)
with pd.ExcelWriter(file_path, mode='a', engine='openpyxl') as writer:
    coord_df.to_excel(writer, index=False, startcol=0, startrow=0)

지리적 좌표: [('도안북로 158 공원 주차장', '36.3317391043375', '127.344830399223'), ('도안북로 117번길 46 앞\n네거리변', '36.3316653798501', '127.341095936769'), ('도안북로117번길 18 와\n도안북로117번길 24 사이', '36.3309804675651', '127.341795622727'), ('대전 서구 도안동 896', '36.3302687680241', '127.341209799864'), ('대전 서구 도안동 962', '36.3299894059564', '127.341208581748'), ('대전 서구 도안동 922', '36.3290685598943', '127.341783707999'), ('대전 서구 도안동 948', '36.3281330632071', '127.341178213471'), ('동서대로 700 과\n대전 서구 도안동 857 사이', None, None), ('대전 서구 도안동 877', '36.3303633575541', '127.339639822247'), ('도안북로93번길 10-15 와\n도안북로93번길 10-11 사이', None, None), ('도안북로93번길 20-11 과\n도안북로93번길 20-17 사이', None, None), ('대전 서구 도안동 846 가운데\n(파스쿠찌와 올리브영 사이)', '36.3309339691769', '127.33862877734'), ('대전 서구 동서대로 688\n(카카오공인중개사 앞 도로변)', '36.3318269660257', '127.339416515096'), ('대전 유성구 용계동 664-6 가운데', '36.3309367419624', '127.337648680487'), ('대전 유성구 용계동 665-5 가운데', '36.3309395067112', '127.3366685835'), ('도안북로 73 과\n동서대로656번길 31-10 사이', None, None), ('동서대

ValueError: Sheet 'Sheet1' already exists and if_sheet_exists is set to 'error'.